<a href="https://colab.research.google.com/github/MohamedDhibi/AI-Based-PMMA-Sheet-Identification-Classification-and-Sorting-Optimization/blob/main/easyocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!git clone https://github.com/MohamedDhibi/AI-Based-PMMA-Sheet-Identification-Classification-and-Sorting-Optimization.git

Cloning into 'AI-Based-PMMA-Sheet-Identification-Classification-and-Sorting-Optimization'...
remote: Enumerating objects: 1122, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1122 (delta 1), reused 1 (delta 0), pack-reused 1116
Receiving objects: 100% (1122/1122), 245.79 MiB | 18.63 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (1102/1102), done.


In [3]:
cd AI-Based-PMMA-Sheet-Identification-Classification-and-Sorting-Optimization

/content/AI-Based-PMMA-Sheet-Identification-Classification-and-Sorting-Optimization


In [4]:
!ls

 Board.pbix		     Dataframe.ipynb	        front
'Classifying images.ipynb'  'Deetecting Shape.ipynb'    ImageCleaning.ipynb
 clean.xlsx		    'Detecting colors.ipynb'   'Missing Data.ipynb'
 color_data.csv		    'Detecting length2.ipynb'   README.md


In [6]:
!pip install pytesseract


In [6]:
!sudo apt-get install tesseract-ocr
!sudo apt-get install tesseract-ocr-eng


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (21.5 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [7]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'


In [1]:
import os
import cv2
import pytesseract
import matplotlib.pyplot as plt
from tqdm import tqdm


In [11]:
# Initialize an empty list to store results
results = []

# Directory containing the images
directory = 'front'  # Replace with your image directory path
# Get the list of image files in the directory
image_files = [f for f in os.listdir(directory) if f.endswith('.jpeg') or f.endswith('.jpg')]

def preprocess_image(image):
    # Resize image to reduce processing time
    resized_image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)  # Adjust fx and fy as needed

    # Convert the resized image to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to preprocess the image
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    return gray, binary

def detect_text(image):
    # Preprocess the image
    gray, binary = preprocess_image(image)

    # Perform OCR on the preprocessed image
    text = pytesseract.image_to_string(binary, config='--psm 3 -l eng --oem 3')  # Using page segmentation mode 3

    # Check if any text is detected
    return bool(text.strip()), text, gray

# Iterate through each file in the directory with tqdm for progress visualization
for filename in tqdm(image_files, desc="Processing images"):
    filepath = os.path.join(directory, filename)

    # Load the current image
    original_image = cv2.imread(filepath)

    # Initialize text detection result
    has_text = False
    detected_text = ""
    gray_image = None

    # Rotate the image from 0 to 315 degrees in steps of 45 degrees
    for angle in range(0, 360, 45):
        rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)
        has_text, detected_text, gray_image = detect_text(rotated_image)

        # Store text in results list if it's longer than 3 characters
        if has_text and len(detected_text) > 3:
            image_id = filename.replace('.jpeg', '').replace('.jpg', '')
            results.append({'ID': image_id, 'text': detected_text})

# Display or save results as needed
results_df = pd.DataFrame(results)

Processing images: 100%|██████████| 1091/1091 [4:15:04<00:00, 14.03s/it]


In [12]:
results_df

,ID,text
0,289,suppeus\n\nwana\n\nvote\n\napis arse:\n\n0\nCr...
1,289,suppeus\n\nwana\n\nvote\n\napis arse:\n\n0\nCr...
2,289,suppeus\n\nwana\n\nvote\n\napis arse:\n\n0\nCr...
3,289,suppeus\n\nwana\n\nvote\n\napis arse:\n\n0\nCr...
4,289,suppeus\n\nwana\n\nvote\n\napis arse:\n\n0\nCr...
...,...,...
579,362,peatanoes ir are ae\n\nbrexicreez\n\n \n\n \n
580,362,peatanoes ir are ae\n\nbrexicreez\n\n \n\n \n
581,362,peatanoes ir are ae\n\nbrexicreez\n\n \n\n \n
582,362,peatanoes ir are ae\n\nbrexicreez\n\n \n\n \n


In [13]:
results_df = results_df.sort_values(by='ID')

In [15]:
results_df.head(10)

,ID,text
11,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
15,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
14,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
8,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
9,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
10,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
13,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
12,100,4 SUTOLXad\n= Spig sos\nok\n\n \n
42,1015,Pibcouses cages\nBaas bra Seppe\n\noo\n\nSrey ...
44,1015,Pibcouses cages\nBaas bra Seppe\n\noo\n\nSrey ...


In [1]:
ls

sample_data/


In [5]:
pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [ ]:
import os
import cv2
import pandas as pd
from tqdm import tqdm
import easyocr

# Initialize EasyOCR reader without specifying a language to capture any text
reader = easyocr.Reader(['en'])  # Empty list for languages to capture any text; GPU usage for faster processing if available

# Directory containing the images
directory = 'front'  # Replace with your image directory path

# Initialize an empty list to store results
results = []

# Supported image extensions
extensions = ['.jpeg', '.jpg']

# Function to rotate the image
def rotate_image(image, angle):
    # Get the image dimensions
    (h, w) = image.shape[:2]
    # Calculate the center of the image
    center = (w / 2, h / 2)
    # Perform the rotation
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h))
    return rotated

# Iterate through all image files in the directory
for filename in tqdm(os.listdir(directory)):
    name, ext = os.path.splitext(filename)
    if ext.lower() in extensions:
        filepath = os.path.join(directory, filename)

        # Load the current image
        image = cv2.imread(filepath)

        # Rotate the image by 0, 90, 180, and 270 degrees
        for angle in tqdm(range(0, 360, 90), desc=f"Rotating {filename}"):
            rotated_image = rotate_image(image, angle)

            # Perform OCR using EasyOCR
            result = reader.readtext(rotated_image)

            # Concatenate all detected texts into one row per image
            concatenated_text = ' '.join([detection[1] for detection in result if len(detection[1]) > 2])

            # Store text in results list for the current image
            results.append({'ID': name, 'text': concatenated_text})

# Display or save results as needed
results_df = pd.DataFrame(results)

Streaming output truncated to the last 5000 lines.
Rotating 544.jpeg:  50%|█████     | 2/4 [00:02<00:02,  1.29s/it]

In [10]:
results_df

,ID,text
0,534,
1,534,
2,534,
3,534,
4,208,
...,...,...
4359,477,
4360,774,
4361,774,
4362,774,


In [14]:
import pandas as pd

# Assuming results_df is your DataFrame

# Convert 'ID' column to integers
results_df['ID'] = pd.to_numeric(results_df['ID'], errors='coerce').astype('Int64')

# Sort rows based on 'ID'
results_df = results_df.sort_values(by='ID')

# Drop rows where 'text' column is empty
results_df = results_df.dropna(subset=['text'])  # Drop rows where 'text' is NaN
results_df = results_df[results_df['text'] != '']  # Drop rows where 'text' is an empty string

# Reset index after sorting and dropping rows
results_df = results_df.reset_index(drop=True)



In [15]:
results_df

,ID,text
0,0,Kem Kemmler Kemmlem Kemmler KemmLeR Kemmier Ke...
1,0,W3h Wam
2,2,) -
3,2,65 - (
4,3,482 PLEXIGLASS XT Weiss 7-IX
...,...,...
894,1077,"aaeretheenanete 7 ,3^=445 .0 OML SVY"
895,1098,"AI. 7""? JM ULZ4 : 0LUUSY W5"
896,1098,"MET8 M 2o ,"
897,1100,W AZ1L ^ QNANHUN % A


In [16]:
# Concatenate 'text' values with a space when 'ID' is the same
results_df = results_df.groupby('ID')['text'].agg(lambda x: ' '.join(x)).reset_index()

In [17]:
results_df

,ID,text
0,0,Kem Kemmler Kemmlem Kemmler KemmLeR Kemmier Ke...
1,2,) - 65 - (
2,3,482 PLEXIGLASS XT Weiss 7-IX PLEX THE ORIGINA ...
3,4,752 12. ~_
4,5,Feb
...,...,...
328,1071,EEE nes tantleerat faeen ete lenter [ai Fea HH...
329,1072,Here Alplaee oxyd Mafat O '93 Eer 3O' EBpl BAb...
330,1077,Ms JHUU Hf Um : HMpHeTU Shanenenenh 7~> aaeret...
331,1098,"AI. 7""? JM ULZ4 : 0LUUSY W5 MET8 M 2o ,"


In [18]:
# Save DataFrame to CSV
results_df.to_csv('important_text.csv', index=False)

print("DataFrame saved as 'important_text.csv'")

DataFrame saved as 'important_text.csv'


In [ ]:
import pandas as pd

# Read CSV file into DataFrame
results_df = pd.read_csv('important_text.csv', dtype={'ID': int, 'text': str})
